In [98]:
import numpy as np 
import pandas as pd
import datetime, csv, os , glob , time
from credential import *

In [20]:
import openai
from openai import OpenAI
print(openai.__version__) #Test if it is working

1.6.1


In [31]:
apikey = credential.api_key

In [33]:
client = OpenAI(api_key=apikey)
#Environment Variable - not working

## Bringing in instruction files: 
- The file is a word document: 'SurveySupportInformation.docx'

In [88]:
#Build a loop for more files to come: 
files_to_upload = ['SurveySupportInformation.docx']

In [89]:
def upload_assistant_file(filename):
    file = client.files.create(
      file=open(filename, "rb"),
      purpose='assistants'
    )
    print(file.id)

In [90]:
for file in files_to_upload:
    upload_assistant_file(file)

file-SEvkSO7e9C720c3MT1EqvAtH


In [91]:
#Loop to confirm the file is uploaded...
for file in client.files.list():
    print(f"{file.filename} has id of: {file.id}")

SurveySupportInformation.docx has id of: file-SEvkSO7e9C720c3MT1EqvAtH


## Adding file(s)' information for the assistant 
(NOT MESSAGE as we are not asking Chat GPT to output something with the data) 


In [92]:
# Add the file to the assistant
assistant = client.beta.assistants.create(
  instructions="You provide information to support user to finish the survey based on the word documents available in your knowledge base.",
  model="gpt-3.5-turbo-1106", # CHECK FOR ERROR IF YOU PROVIDE THE WRONG MODEL
  tools=[{"type": "retrieval"}],
  file_ids= ['file-SEvkSO7e9C720c3MT1EqvAtH'])

## Run the survey

In [93]:
thread = client.beta.threads.create()

In [94]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="How many questions are there in the survey, can you list them all?"
)

In [95]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [96]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id,
)
print(run.status)

in_progress


In [99]:
def display_thread_messages(messages):
    # EXPECTS MESSAGES IN ASC ORDER!
    for thread_message in messages.data:
        print(thread_message.content[0].text.value)
        print('\n')
        
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(1)
    return run
run = wait_on_run(run,thread)

In [100]:
run.status

'completed'

In [101]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc",
)

In [102]:
for message in messages:
    print(message)
    print('\n')

ThreadMessage(id='msg_j7xoKAza2YRhKF7jqPe04oPx', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='How many questions are there in the survey, can you list them all?'), type='text')], created_at=1703807010, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_EHqnBPF9D3b3aWoLCMrjlqo1')


ThreadMessage(id='msg_FRNYQnH7lzBHgc75sunExu0X', assistant_id='asst_jrNI0BPCd9CqS1rEm8o8B6Ev', content=[MessageContentText(text=Text(annotations=[], value="The survey contains six questions, each with multiple choice responses. Here are the questions listed:\n\n1. Stable Income: choose between\n   - Our household income is stable\n   - Our household income doesn’t change that much\n   - Our household income always goes up and down and we can’t plan ahead.\n\n2. Access to Gas/Electricity: choose between\n   - We can afford to pay for gas and/or electricity of all of the time.\n   - We have gas and/or electricity but we can't use it 

In [103]:
display_thread_messages(messages)

How many questions are there in the survey, can you list them all?


The survey contains six questions, each with multiple choice responses. Here are the questions listed:

1. Stable Income: choose between
   - Our household income is stable
   - Our household income doesn’t change that much
   - Our household income always goes up and down and we can’t plan ahead.

2. Access to Gas/Electricity: choose between
   - We can afford to pay for gas and/or electricity of all of the time.
   - We have gas and/or electricity but we can't use it all the time because of the cost.
   - In the past year sometimes there has been no gas or electricity and we have had to choose between cooking or heating.

3. Communication & social capital: choose between
   - We mix with many people who are a good influence on us. We use these connections to open up opportunities.
   - We mix with people but we don't make the most of it.
   - In our household we don't mix much and those people we know have a bad inf

In [104]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can I have the question in Chinese? Also for question 1, I work on zero hour contract, is that means I have unstable income?"
)

In [105]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [106]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id,
)
print(run.status)

in_progress


In [107]:
run = wait_on_run(run,thread)

In [108]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc",
)

In [109]:
display_thread_messages(messages)

How many questions are there in the survey, can you list them all?


The survey contains six questions, each with multiple choice responses. Here are the questions listed:

1. Stable Income: choose between
   - Our household income is stable
   - Our household income doesn’t change that much
   - Our household income always goes up and down and we can’t plan ahead.

2. Access to Gas/Electricity: choose between
   - We can afford to pay for gas and/or electricity of all of the time.
   - We have gas and/or electricity but we can't use it all the time because of the cost.
   - In the past year sometimes there has been no gas or electricity and we have had to choose between cooking or heating.

3. Communication & social capital: choose between
   - We mix with many people who are a good influence on us. We use these connections to open up opportunities.
   - We mix with people but we don't make the most of it.
   - In our household we don't mix much and those people we know have a bad inf

# Costly but trying to loop! 

In [111]:
question = '' 
print("ASSISTENT CONNECTED, TYPE 'BYE' TO END THE CONVERSATION")

def display_thread_messages(messages):
    # EXPECTS MESSAGES IN ASC ORDER!
    for thread_message in messages.data:
        print(thread_message.content[0].text.value)
        print('\n')
        
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(1)
    return run

while question != 'BYE':
    question = input("")
    message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = question)
    run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,)
    run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id,)
    run = wait_on_run(run,thread)
    messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc",)
    display_thread_messages(messages)
    

ASSISTENT CONNECTED, TYPE 'BYE' TO END THE CONVERSATION


 What are the questions of the survey


How many questions are there in the survey, can you list them all?


The survey contains six questions, each with multiple choice responses. Here are the questions listed:

1. Stable Income: choose between
   - Our household income is stable
   - Our household income doesn’t change that much
   - Our household income always goes up and down and we can’t plan ahead.

2. Access to Gas/Electricity: choose between
   - We can afford to pay for gas and/or electricity of all of the time.
   - We have gas and/or electricity but we can't use it all the time because of the cost.
   - In the past year sometimes there has been no gas or electricity and we have had to choose between cooking or heating.

3. Communication & social capital: choose between
   - We mix with many people who are a good influence on us. We use these connections to open up opportunities.
   - We mix with people but we don't make the most of it.
   - In our household we don't mix much and those people we know have a bad inf

 On Dental Care, what should I choose if I go to dentist every year but my wife havent been for 6 years 


How many questions are there in the survey, can you list them all?


The survey contains six questions, each with multiple choice responses. Here are the questions listed:

1. Stable Income: choose between
   - Our household income is stable
   - Our household income doesn’t change that much
   - Our household income always goes up and down and we can’t plan ahead.

2. Access to Gas/Electricity: choose between
   - We can afford to pay for gas and/or electricity of all of the time.
   - We have gas and/or electricity but we can't use it all the time because of the cost.
   - In the past year sometimes there has been no gas or electricity and we have had to choose between cooking or heating.

3. Communication & social capital: choose between
   - We mix with many people who are a good influence on us. We use these connections to open up opportunities.
   - We mix with people but we don't make the most of it.
   - In our household we don't mix much and those people we know have a bad inf

 BYE


How many questions are there in the survey, can you list them all?


The survey contains six questions, each with multiple choice responses. Here are the questions listed:

1. Stable Income: choose between
   - Our household income is stable
   - Our household income doesn’t change that much
   - Our household income always goes up and down and we can’t plan ahead.

2. Access to Gas/Electricity: choose between
   - We can afford to pay for gas and/or electricity of all of the time.
   - We have gas and/or electricity but we can't use it all the time because of the cost.
   - In the past year sometimes there has been no gas or electricity and we have had to choose between cooking or heating.

3. Communication & social capital: choose between
   - We mix with many people who are a good influence on us. We use these connections to open up opportunities.
   - We mix with people but we don't make the most of it.
   - In our household we don't mix much and those people we know have a bad inf

## Important Cost Saving: Delete Assistant and All Files

In [ ]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
response = client.beta.assistants.delete(my_assistants.data[0].id)
print(response)

In [ ]:
for file in client.files.list():
    client.files.delete(file.id)

In [ ]:
client.files.list()